# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator.graph_representation_generator import (
    GraphRepresentationGenerator,
)
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.graph_prompter_hf.classifier import GraphPrompterHF

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
EPOCHS = 2
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data.to("cuda"),
    kg_manager.gnn_train_data.to("cuda"),
    kg_manager.gnn_val_data.to("cuda"),
    kg_manager.gnn_test_data.to("cuda"),
    hidden_channels=128,
    kge_dimension=128,
    force_recompute=False,
    device="cuda",
)

loading pretrained model
Device: 'cuda'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
graph_prompter_hf_bert_classifier = GraphPrompterHF(
    kge_manager=kg_manager,
    get_embeddings_cb=graph_representation_generator_graph_prompter_hf.get_embeddings,
    model_name=MODEL_NAME,
    vanilla_model_path=f"{ROOT}/llm/graph_prompter_hf_frozen/training/best",
    gnn_parameters=list(
        graph_representation_generator_graph_prompter_hf.model.parameters()
    ),
)

device cuda
6
6
6


In [10]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
)

Loading dataset from disk:   0%|          | 0/127 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/46 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/46 [00:00<?, ?it/s]

In [11]:
graph_prompter_hf_bert_classifier.model.device

device(type='cuda', index=0)

In [12]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE_LLM,
)
graph_representation_generator_graph_prompter_hf.save_model()

Adding GNN parameters to optimizer


  0%|          | 0/140002 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.0628, 'grad_norm': 0.42682206630706787, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.0666, 'grad_norm': 0.4896775782108307, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.0692, 'grad_norm': 0.5148248076438904, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.0648, 'grad_norm': 0.3533174991607666, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.0791, 'grad_norm': 0.5547524094581604, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.0641, 'grad_norm': 0.39158710837364197, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.0711, 'grad_norm': 0.5112154483795166, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.0756, 'grad_norm': 0.6109243035316467, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.0695, 'grad_norm': 0.5231167674064636, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.053, 'grad_norm': 0.255573570728302, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.0794, 'grad_norm': 0.382215

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.08857127279043198, 'eval_accuracy': 0.970030812307423, 'eval_runtime': 17239.6717, 'eval_samples_per_second': 371.239, 'eval_steps_per_second': 1.45, 'epoch': 1.0}
{'loss': 0.0772, 'grad_norm': 0.3589099645614624, 'learning_rate': 2.5086378689911257e-05, 'epoch': 1.0}
{'loss': 0.0693, 'grad_norm': 0.35600703954696655, 'learning_rate': 2.5082794511906642e-05, 'epoch': 1.0}
{'loss': 0.0755, 'grad_norm': 0.28887224197387695, 'learning_rate': 2.5079210333902027e-05, 'epoch': 1.0}
{'loss': 0.0577, 'grad_norm': 0.285454124212265, 'learning_rate': 2.507562615589741e-05, 'epoch': 1.0}
{'loss': 0.0836, 'grad_norm': 0.35966840386390686, 'learning_rate': 2.507204197789279e-05, 'epoch': 1.0}
{'loss': 0.0652, 'grad_norm': 0.42158377170562744, 'learning_rate': 2.5068457799888172e-05, 'epoch': 1.0}
{'loss': 0.0896, 'grad_norm': 0.5937926173210144, 'learning_rate': 2.506487362188356e-05, 'epoch': 1.0}
{'loss': 0.0675, 'grad_norm': 0.328003466129303, 'learning_rate': 2.5061289443878942e

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.08484328538179398, 'eval_accuracy': 0.971449553440291, 'eval_runtime': 17888.4019, 'eval_samples_per_second': 357.776, 'eval_steps_per_second': 1.398, 'epoch': 2.0}
{'train_runtime': 142590.8428, 'train_samples_per_second': 251.35, 'train_steps_per_second': 0.982, 'train_loss': 0.07847921369206086, 'epoch': 2.0}


In [ ]:
import torch

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if (tens_1[key] == tens_2[key]).all():
            print(key, "is same")
        else:
            print(key, "is not same")

target_lin.weight is not same
target_lin.bias is not same
source_emb.weight is not same
target_emb.weight is not same
gnn.conv1.source__edge__target.lin_l.weight is not same
gnn.conv1.source__edge__target.lin_l.bias is not same
gnn.conv1.source__edge__target.lin_r.weight is not same
gnn.conv1.target__rev_edge__source.lin_l.weight is not same
gnn.conv1.target__rev_edge__source.lin_l.bias is not same
gnn.conv1.target__rev_edge__source.lin_r.weight is not same
gnn.conv2.source__edge__target.lin_l.weight is not same
gnn.conv2.source__edge__target.lin_l.bias is not same
gnn.conv2.source__edge__target.lin_r.weight is not same
gnn.conv2.target__rev_edge__source.lin_l.weight is not same
gnn.conv2.target__rev_edge__source.lin_l.bias is not same
gnn.conv2.target__rev_edge__source.lin_r.weight is not same
